# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

file_path_list = []

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list += glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a full event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')

### Create tables to run the following queries.

## Create queries to ask the following three questions of the data

### 1. Get the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Get only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [26]:
## -----------------------------------------------------------------------------------------------------------------------------------------------------------

## Task 1: Getting the artist song title and song length for sessionid=338 and itemInSession=4

In [27]:
### For this Query 1 itemInSession and sessionId are considered as composite key because we use sessionId as sessionId to group the data in nodes evenly and sort it by itemInSession a condition which we use in the WHERE clause

In [8]:
#create and execute table song_by_user_on_session statement creation
query_one = "CREATE TABLE IF NOT EXISTS song_details_for_itemInSession"
query_one = query_one + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY ((sessionId), itemInSession))"
try:
    session.execute(query_one)
except Exception as e:
    print(e)                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
#insert data from  file in table song_details_for_itemInSession
        query_one = "INSERT INTO song_details_for_itemInSession (sessionId, itemInSession, artist, song, length)"
        query_one = query_one + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query_one, ( int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
query = "select artist, song, length from song_details_for_itemInSession WHERE sessionid=338 and itemInSession=4;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Task 2: Get name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [28]:
### For this Query userId and itemInSession are considered as composite key because we use userId as partition key and itemInSession as clustering column to sort the songs based on itemInSession

In [16]:
#create and execute table song_by_user_on_session statement creation
query_two = "CREATE TABLE IF NOT EXISTS song_by_user_on_session"
query_two = query_two + "(userId int, itemInSession int, sessionId int, artist text, song text, length float, firstName text, lastName text,  PRIMARY KEY ((userId), itemInSession))"
try:
    session.execute(query_two)
except Exception as e:
    print(e)         

In [17]:
file = 'event_datafile_new.csv'
#insert data from  file in table song_by_user_on_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query_one = "INSERT INTO song_by_user_on_session ( userId, itemInSession, sessionId, artist, song, length, firstName, lastName)"
        query_one = query_one + "VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query_one, (int(line[10]), int(line[3]), int(line[8]), line[0], line[9], float(line[5]), line[1], line[4]))

In [18]:
query = "select artist, song, firstName, lastName, itemInSession from song_by_user_on_session WHERE userid = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname, row.iteminsession)   

Down To The Bone Keep On Keepin' On Sylvie Cruz 0
Three Drives Greece 2000 Sylvie Cruz 1
Sebastien Tellier Kilometer Sylvie Cruz 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3


## Task 3: Get user (first and last name) for song = 'All Hands Against His Own'

In [29]:
### For this Query song and userId are considered as composite key because we use song as partition key and userId as clustering column to uniquely identify each record per song per user

In [14]:
#create and execute table user_list_for_song statement creation
query_three = "CREATE TABLE IF NOT EXISTS user_list_for_song"
query_three = query_three + "(song text, userId int, firstName text, lastName text, PRIMARY KEY ((song), userId))"
try:
    session.execute(query_three)
except Exception as e:
    print(e)                

In [15]:
#insert data from  file in table user_list_for_song
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query_one = "INSERT INTO user_list_for_song (song, userId, firstName, lastName)"
        query_one = query_one + "VALUES(%s, %s, %s, %s)"
        session.execute(query_one, (line[9], int(line[10]), line[1], line[4]))

In [34]:
#query data in table user_list_for_song
query = "select firstname, lastname from user_list_for_song WHERE song ='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
#Drop the tables before closing out the sessions

In [1]:
query_drop = "DROP TABLE query_library_one"
try:
    session.execute(query_drop)
except Exception as e:
    print(e)  
query_drop = "DROP TABLE query_library_two"
try:
    session.execute(query_drop)
except Exception as e:
    print(e)  
query_drop = "DROP TABLE query_library_three"
try:
    session.execute(query_drop)
except Exception as e:
    print(e)  

name 'session' is not defined
name 'session' is not defined
name 'session' is not defined


### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()